This notebook should analyse bell inequalities that are found for the finite efficiency PR box. The bell inequalities
considered here should equalize at the threshold efficiency of the PR box.

We want to find out if there is a structure behind these bell inequalities.

In [49]:
import numpy as np
from linearbell.utils import get_configs
file_2233 = '../data/vertex_enum_pr_box_det_decomp/2233.gz'
file_3333 = '../data/vertex_enum_pr_box_det_decomp/3333.gz'

bell2233 = np.loadtxt(file_2233)
bell3333 = np.loadtxt(file_3333)

configs2233 = get_configs(range(2), range(2), range(3), range(3))
configs3333 = get_configs(range(3), range(3), range(3), range(3))

We have now loaded the bell inequalities and want to display them in matrix form. Currently they are just in vector
form and the configuration of inputs / outputs at each entry is stored in the corresponding *configXXXX* variable.
Therefore define a function that returns the configurations in matrix form and a function that transforms vector form
bell inequalities to matrix form.

In [50]:
from itertools import product
def get_configs_mat(inputs_a, inputs_b, outputs_a, outputs_b):
    """ Returns the configurations in a matrix form """
    configs = []
    for y,b in product(inputs_b, outputs_b):
        c = []
        for x, a in product(inputs_a, outputs_a):
            c.append((a,b,x,y))
        configs.append(c)
    return configs

def transform_vec_to_mat(configs_vec, configs_mat, vec):
    """ Transforms a vector to matrix representation """
    mat = np.zeros((len(configs_mat), len(configs_mat[0])))
    for l,c in enumerate(configs_vec):
        for i in range(len(configs_mat)):
            try:
                j = configs_mat[i].index(c)
                # if not fails -> set the entry in matrix
                mat[i,j] = vec[l]
            except ValueError as e:
                pass
    return mat



Now use these functions to reshape the vectors of bell inequalities

In [51]:
configs2233_mat = get_configs_mat(range(2), range(2), range(3), range(3))
configs3333_mat = get_configs_mat(range(3), range(3), range(3), range(3))

bell2233_mat = transform_vec_to_mat(configs2233, configs2233_mat, bell2233)
bell3333_mat = transform_vec_to_mat(configs3333, configs3333_mat, bell3333)

assert np.sum(bell2233) == np.sum(bell2233_mat)
assert np.sum(bell3333) == np.sum(bell3333_mat)

The forms of the bell inequalities look weird, so lets check if the conversion is correct

In [52]:
for l, (a,b,x,y) in enumerate(configs2233):
    if bell2233[l] > 0:
        print('a: {} | b: {} | x: {} | y: {}'.format(a,b,x,y))



a: 0 | b: 0 | x: 0 | y: 0
a: 0 | b: 1 | x: 0 | y: 0
a: 0 | b: 2 | x: 0 | y: 0
a: 1 | b: 0 | x: 0 | y: 0
a: 1 | b: 1 | x: 0 | y: 0
a: 1 | b: 2 | x: 0 | y: 0
a: 2 | b: 0 | x: 0 | y: 0
a: 2 | b: 1 | x: 0 | y: 0
a: 2 | b: 2 | x: 0 | y: 0


Conversion seems to be working. Only the entries for $x=0=y$ are non zero in the bell expression.

Check if these bell inequalities actually equalize the PR box at threshold efficiency.

In [53]:
from linearbell.utils import general_pr_box_extended
eta2 = 4 / (4 + 2)
pr_box2233 = [general_pr_box_extended(a, b, x, y, eta2, range(2)) for (a, b, x, y) in configs2233]
pr_box2233 = np.array(pr_box2233)

assert pr_box2233 @ bell2233 == 1.0

epsilon = -0.1
pr_box2233_high = [general_pr_box_extended(a, b, x, y, eta2 + epsilon, range(2)) for (a, b, x, y) in configs2233]
pr_box2233_high = np.array(pr_box2233_high)

assert pr_box2233_high @ bell2233 > 1.0, pr_box2233_high @ bell2233

AssertionError: 1.0

So the bell inequality is not a bell inequality actually. It's just summing up all
probabilities for one input pair, which is of course 1 due to normalization.